In [68]:
import sympy as sp

In [69]:
num_poles = 3

pole_as = [sp.Symbol(f'a_{i}') for i in range(num_poles)]
pole_ls = [sp.Symbol(f'l_{i}') for i in range(num_poles)]
pole_ms = [sp.Symbol(f'm_{i}') for i in range(num_poles)]
pole_ds = [sp.Symbol(f'd_{i}') for i in range(num_poles)]
pole_Js = [sp.Symbol(f'J_{i}') for i in range(num_poles)]

m_c = sp.Symbol("m_c")
g = sp.Symbol("g")

In [70]:
s = sp.Symbol("s")
d_s = sp.Symbol("\\dot{s}")
dd_s = sp.Symbol("\\ddot{s}")
thetas = [sp.Symbol(f"\\theta_{i+1}") for i in range(num_poles)]
d_thetas = [sp.Symbol(f"\\dot{{\\theta}}_{i+1}") for i in range(num_poles)]
dd_thetas = [sp.Symbol(f"\\ddot{{\\theta}}_{i+1}") for i in range(num_poles)]
u = dd_s

derivate_by_time_dict = {s: d_s, d_s: dd_s}
for i in range(num_poles):
    derivate_by_time_dict[thetas[i]] = d_thetas[i]
    derivate_by_time_dict[d_thetas[i]] = dd_thetas[i]

In [72]:
pole_pcs = []
for i, (theta, a) in enumerate(zip(thetas, pole_as)):
    prev_1 = 0
    prev_2 = 0
    for prev_l, prev_theta in list(zip(pole_ls, thetas))[:i]:
        prev_1 += -prev_l*sp.sin(prev_theta)
        prev_2 += prev_l*sp.cos(prev_theta)
    pole_pcs.append([s-a*sp.sin(theta)+prev_1, a*sp.cos(theta)+prev_2])

In [73]:
def square_sum(x: tuple):
    result = 0
    for x_i in x:
        result += x_i**2
    return result

T_c = 1/2*m_c*d_s**2
T_p = 1/2*sum(m*square_sum(pc) for m, pc in zip(pole_ms, pole_pcs))+1/2*sum(J*d_theta**2 for J, d_theta in zip(pole_Js, d_thetas))
T = T_c + T_p
V = 0
for m, pc in zip(pole_ms, pole_pcs):
    V += g*m*pc[0]
R = 0
prev_w = 0
for d, d_theta in zip(pole_ds, d_thetas):
    R += 1/2*d*(d_theta-prev_w)**2
    prev_w = d_theta

In [74]:
L = T-V
eqs = []
lh = sp.diff(L, d_s).subs(derivate_by_time_dict)
lh -= sp.diff(L, s) + sp.diff(R, d_s)
rh = u
eqs.append(lh-rh)
for theta, d_theta in zip(thetas, d_thetas):
    lh = sp.diff(L, d_theta).subs(derivate_by_time_dict)
    lh -= sp.diff(L, theta) + sp.diff(R, d_theta)
    rh = 0
    eqs.append(lh-rh)
eqs

[1.0*\ddot{s}*m_c - \ddot{s} + g*m_0 + g*m_1 + g*m_2 - 0.5*m_0*(-2*a_0*sin(\theta_1) + 2*s) - 0.5*m_1*(-2*a_1*sin(\theta_2) - 2*l_0*sin(\theta_1) + 2*s) - 0.5*m_2*(-2*a_2*sin(\theta_3) - 2*l_0*sin(\theta_1) - 2*l_1*sin(\theta_2) + 2*s),
 1.0*J_0*\ddot{\theta}_1 - 1.0*\dot{\theta}_1*d_0 - a_0*g*m_0*cos(\theta_1) - 0.5*d_1*(2*\dot{\theta}_1 - 2*\dot{\theta}_2) - g*l_0*m_1*cos(\theta_1) - g*l_0*m_2*cos(\theta_1) - 0.5*m_0*(-2*a_0**2*sin(\theta_1)*cos(\theta_1) - 2*a_0*(-a_0*sin(\theta_1) + s)*cos(\theta_1)) - 0.5*m_1*(-2*l_0*(a_1*cos(\theta_2) + l_0*cos(\theta_1))*sin(\theta_1) - 2*l_0*(-a_1*sin(\theta_2) - l_0*sin(\theta_1) + s)*cos(\theta_1)) - 0.5*m_2*(-2*l_0*(a_2*cos(\theta_3) + l_0*cos(\theta_1) + l_1*cos(\theta_2))*sin(\theta_1) - 2*l_0*(-a_2*sin(\theta_3) - l_0*sin(\theta_1) - l_1*sin(\theta_2) + s)*cos(\theta_1)),
 1.0*J_1*\ddot{\theta}_2 - a_1*g*m_1*cos(\theta_2) - 0.5*d_1*(-2*\dot{\theta}_1 + 2*\dot{\theta}_2) - 0.5*d_2*(2*\dot{\theta}_2 - 2*\dot{\theta}_3) - g*l_1*m_2*cos(\thet

In [86]:
sol = sp.solve(eqs, dd_thetas+[dd_s])
sp.simplify(sol[dd_thetas[0]])

(-\dot{\theta}_2*d_2 + \dot{\theta}_3*d_2 + a_2*g*m_2*cos(\theta_3) + a_2*l_0*m_2*sin(\theta_1 - \theta_3) + a_2*l_1*m_2*sin(\theta_2 - \theta_3) - a_2*m_2*s*cos(\theta_3))/J_2